[View in Colaboratory](https://colab.research.google.com/github/nicewook/datascience_exercise/blob/master/timelog_to_srt.ipynb)

# 상황

### 동영상이 있고
### 동영상의 상황에서 특정 이벤트 발생시점을 기록한 time 정보가 있다. 


# 목표

### 이걸로 자막파일을 만들어 동영상을 보면서 자막을 통해 이벤트 발생을 확인할 수 있도록 한다. 

# 분석을 위한 영상 참고자료

- 이 경우는 특정 차량이 지나간 시점을 확인하려고 한다.

## Lane 2 분석

### 통계 시작 : 15:00:04 (검정색 말리부 1872) / (CCTV 10:03초 부근)
- 앞 차량 : 검정 소나타
- 뒤 차량 : 검정 SUV

 
### 통계 종료 : 15:21:34 (흰색 투싼 4052) / (CCTV 31:32초 부근)
- 앞 차량 : 파란색 1톤 포터 (다수의 박스가 화물칸에 있음)
- 뒤 차량 : 흰색 K5

# 로컬PC의 파일을 업로드하자

### 아래 셀을 실행하면 ** 파일선택 ** 버튼이 나오며 이를 통해 파일을 업로드 가능하다

In [3]:
from google.colab import files
uploaded = files.upload() # 파일 업로드 기능 실행

for fn in uploaded.keys(): # 업로드된 파일 정보 출력
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

Saving 데이터비교v3_2차로_vdu.txt to 데이터비교v3_2차로_vdu.txt
User uploaded file "데이터비교v3_2차로_vdu.txt" with length 12425 bytes


# 파일을 pandas read_csv 를 이용해 가져온다

In [198]:
ls -al

total 108
drwxr-xr-x 1 root root  4096 Jun  2 02:17 ./
drwxr-xr-x 1 root root  4096 Jun  1 23:30 ../
drwx------ 4 root root  4096 Jun  1 23:31 .cache/
drwxr-xr-x 3 root root  4096 Jun  1 23:31 .config/
drwxr-xr-x 1 root root  4096 May 21 17:02 datalab/
drwxr-xr-x 4 root root  4096 Jun  1 23:31 .forever/
drwxr-xr-x 5 root root  4096 Jun  1 23:31 .ipython/
-rw-r--r-- 1 root root 15297 Jun  2 02:21 lane2_1.csv
drwx------ 3 root root  4096 Jun  1 23:31 .local/
-rw------- 1 root root  1024 Jun  1 23:31 .rnd
-rw-r--r-- 1 root root 39544 Jun  2 02:21 데이터비교v3_2차로_vdu.srt
-rw-r--r-- 1 root root 12425 Jun  1 23:32 데이터비교v3_2차로_vdu.txt


In [241]:
import pandas as pd

data = pd.read_csv('데이터비교v3_2차로_vdu.txt', header=None, engine='python')  # 경로나 파일명에 한글이 있을경우 engine 옵션 넣어줘야함

data.columns = ['time']  # 컬럼명을 정해줌

data.time = data.time.astype('str')
data["subtitle"] = data.time.str[11:]  # 나중에 자막으로 쓰기위해 빼두자

data.head() # 데이터 내용을 조금 보자

,time,subtitle
0,2018.05.23 15:00:03.828,15:00:03.828
1,2018.05.23 15:00:05.871,15:00:05.871
2,2018.05.23 15:00:07.252,15:00:07.252
3,2018.05.23 15:00:08.124,15:00:08.124
4,2018.05.23 15:00:13.072,15:00:13.072


# time 정보를 동영상이 시작되고 자막이 보여질 시점으로 맞춰보자



In [242]:
data.time = pd.to_datetime(data.time)  # string 에서 datetime type 으로 변경
data.head()

,time,subtitle
0,2018-05-23 15:00:03.828,15:00:03.828
1,2018-05-23 15:00:05.871,15:00:05.871
2,2018-05-23 15:00:07.252,15:00:07.252
3,2018-05-23 15:00:08.124,15:00:08.124
4,2018-05-23 15:00:13.072,15:00:13.072


In [243]:
type(data.time[0])  # Timestamp 타입이라고 나온다

pandas._libs.tslib.Timestamp

# 시간정보중 불필요한 시간은 날려버림
- 15:00:04 부터 15:21:34 까지만 보면 되므로 
- 2018-05-23 15:21:35.000' 보다 이전인 데이터만 필요함
- 현재 데이터는 딱 이부분만 있으므로 원래는 할 필요가 없는 작업임


### 통계 시작 : 15:00:04 (검정색 말리부 1872) / (CCTV 10:03초 부근)
앞 차량 : 검정 소나타
뒤 차량 : 검정 SUV
### 통계 종료 : 15:21:34 (흰색 투싼 4052) / (CCTV 31:32초 부근)
앞 차량 : 파란색 1톤 포터 (다수의 박스가 화물칸에 있음)
뒤 차량 : 흰색 K5

In [244]:
print(len(data))
data = data.loc[data['time'] <'2018-05-23 15:21:35.000']
print(len(data))

497
497


### 우선은 시작시간으로 전체 시간을 빼준다. 그러면 00:00:00.0000 에서 시작하게 된다. 
### 여기에 자막의 첫 시작 시간인 10:03:0000 을  더해주면 자막시작시간에 맞춰진다. 

- 동영상의 재생시간과 실제 날짜, 시간의 정확한 매칭을 알면 더욱 정확하게 설정할 수 있다
- 즉, 실제시간이 15:00:03.8280 이 동영상 재생시간의 00:10:03.4231 이라는 식의 정보가 필요하다


In [245]:
data.time = data.time - data.time[0]  # 가장 첫 시간으로 빼버리면 첫 자막의 시작시간은 0이 되겠지? 
data.time = data.time + pd.Timedelta('00:10:03')
data.head()

,time,subtitle
0,00:10:03,15:00:03.828
1,00:10:05.043000,15:00:05.871
2,00:10:06.424000,15:00:07.252
3,00:10:07.296000,15:00:08.124
4,00:10:12.244000,15:00:13.072


### 원하는 형식으로 바꿔보자
1. pandas TimeDelta 형식을 datetime 형식으로 바꾸고
2. string format 을 srt 자막파일이 요구하는 시간으로 바꾼다. 
3. 자막파일은 0.1 milliseconds 까지 표시하므로 마지막 두 자리는 버린다

In [246]:
type(data.time[0]) 

pandas._libs.tslib.Timedelta

In [247]:
data['time'] = pd.to_datetime(data['time']).dt.strftime("%H:%M:%S.%f").str[:-2]
print(type(data.time[0]))
data.head()

<class 'str'>


,time,subtitle
0,00:10:03.0000,15:00:03.828
1,00:10:05.0430,15:00:05.871
2,00:10:06.4240,15:00:07.252
3,00:10:07.2960,15:00:08.124
4,00:10:12.2440,15:00:13.072


# 중간저장 
### 인덱스를 reset 해주고 저장하자

In [248]:
data.reset_index(drop=True, inplace=True)
print(data.head())

            time      subtitle
0  00:10:03.0000  15:00:03.828
1  00:10:05.0430  15:00:05.871
2  00:10:06.4240  15:00:07.252
3  00:10:07.2960  15:00:08.124
4  00:10:12.2440  15:00:13.072


In [0]:
data.to_csv('lane2_1.csv', header=None)

# srt 자막을 만들어보자

### 자막이 다음 자막이 시작되기 전까지 보여지도록 하자
- 따라서 한 자막의 종료 시간은 다음 자막의 시작시간과 일치한다


In [250]:
data_endtime = data.iloc[1:]

# 당연히 data2 의 길이가 하나 짧다
print(len(data))
print(len(data_endtime))

497
496


In [251]:
# 맨 data_endtime 종료시간을 추가해주자
data_endtime = data_endtime.append(data_endtime.iloc[-1:].copy())
temp = data_endtime.iloc[-1].time
temp = temp[:7] + "5" + temp[8:]
data_endtime.iloc[-1].time = temp
print(len(data_endtime))
data_endtime.tail()

497


,time,subtitle
493,00:31:24.8890,15:21:25.717
494,00:31:26.3010,15:21:27.129
495,00:31:28.9050,15:21:29.733
496,00:31:32.1000,15:21:32.928
496,00:31:35.1000,15:21:32.928


In [252]:
# index 재정렬
data_endtime.reset_index(drop=True, inplace=True)
print(len(data_endtime))
data_endtime.tail()

497


,time,subtitle
492,00:31:24.8890,15:21:25.717
493,00:31:26.3010,15:21:27.129
494,00:31:28.9050,15:21:29.733
495,00:31:32.1000,15:21:32.928
496,00:31:35.1000,15:21:32.928


### srt 자막 형식은 인터넷 검색을 참고할 것

### 일단은 자막이 재생될 시간정보를 생성하자

In [253]:
# make start string
data.time = data.time.astype('str')
data_endtime.time = data_endtime.time.astype('str')

data.time = data.time.str.replace('.', ',')
data_endtime.time = data_endtime.time.str.replace('.', ',')

data['time'] = data.time \
            + " --> " \
            + data_endtime.time

data.head()


00:10:03,0000 --> 00:10:05,0430
                              time      subtitle
0  00:10:03,0000 --> 00:10:05,0430  15:00:03.828
1  00:10:05,0430 --> 00:10:06,4240  15:00:05.871
2  00:10:06,4240 --> 00:10:07,2960  15:00:07.252
3  00:10:07,2960 --> 00:10:12,2440  15:00:08.124
4  00:10:12,2440 --> 00:10:13,5650  15:00:13.072


### 그리고 실제로 보여질 자막시간 정보
  - 일단은 차량 대수가 1, 2, 3 증가하게 하고
  - 그 뒤로 차량 인식 시간을 넣어주자 

In [254]:
data["car_num"] = data.index.astype(int) + 1
data.subtitle = "car count: " + data.car_num.astype('str') + " - " + data.subtitle.astype('str')
data.head()

,time,subtitle,car_num
0,"00:10:03,0000 --> 00:10:05,0430",car count: 1 - 15:00:03.828,1
1,"00:10:05,0430 --> 00:10:06,4240",car count: 2 - 15:00:05.871,2
2,"00:10:06,4240 --> 00:10:07,2960",car count: 3 - 15:00:07.252,3
3,"00:10:07,2960 --> 00:10:12,2440",car count: 4 - 15:00:08.124,4
4,"00:10:12,2440 --> 00:10:13,5650",car count: 5 - 15:00:13.072,5


In [255]:
srt_data = data[['car_num', 'time', 'subtitle']].copy()
srt_data.car_num = srt_data.car_num.astype('str')
srt_data.time = srt_data.time.astype('str')
srt_data.subtitle = srt_data.subtitle.astype('str')

srt_data["srt"] = srt_data.car_num.str[:] + "\n" \
                  + srt_data.time.str[:] + "\n" \
                  + srt_data.subtitle.str[:] + "\n\n"
srt_data.head()

,car_num,time,subtitle,srt
0,1,"00:10:03,0000 --> 00:10:05,0430",car count: 1 - 15:00:03.828,"1\n00:10:03,0000 --> 00:10:05,0430\ncar count:..."
1,2,"00:10:05,0430 --> 00:10:06,4240",car count: 2 - 15:00:05.871,"2\n00:10:05,0430 --> 00:10:06,4240\ncar count:..."
2,3,"00:10:06,4240 --> 00:10:07,2960",car count: 3 - 15:00:07.252,"3\n00:10:06,4240 --> 00:10:07,2960\ncar count:..."
3,4,"00:10:07,2960 --> 00:10:12,2440",car count: 4 - 15:00:08.124,"4\n00:10:07,2960 --> 00:10:12,2440\ncar count:..."
4,5,"00:10:12,2440 --> 00:10:13,5650",car count: 5 - 15:00:13.072,"5\n00:10:12,2440 --> 00:10:13,5650\ncar count:..."


In [256]:
srt_data = srt_data.srt
srt_data.head()

0    1\n00:10:03,0000 --> 00:10:05,0430\ncar count:...
1    2\n00:10:05,0430 --> 00:10:06,4240\ncar count:...
2    3\n00:10:06,4240 --> 00:10:07,2960\ncar count:...
3    4\n00:10:07,2960 --> 00:10:12,2440\ncar count:...
4    5\n00:10:12,2440 --> 00:10:13,5650\ncar count:...
Name: srt, dtype: object

# 마지막으로 저장하자

In [257]:

file_name = "데이터비교v3_2차로_vdu.srt"
srt_data.to_csv(file_name, header=None, mode='w', index=False)
print(len(srt_data))

497


In [258]:
!head 데이터비교v3_2차로_vdu.srt

"1
00:10:03,0000 --> 00:10:05,0430
car count: 1 - 15:00:03.828

"
"2
00:10:05,0430 --> 00:10:06,4240
car count: 2 - 15:00:05.871

"


# 쌍따옴표 제거만 하면 끝

###아쉽게도 이유는 모르겠지만. 쌍따옴표들이 생겨있다. 

In [0]:
# Read in the file
with open(file_name, 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('"', '')

# Write the file out again
with open(file_name, 'w') as file:
  file.write(filedata)

In [260]:
!head 데이터비교v3_2차로_vdu.srt

1
00:10:03,0000 --> 00:10:05,0430
car count: 1 - 15:00:03.828


2
00:10:05,0430 --> 00:10:06,4240
car count: 2 - 15:00:05.871




# 마지막으로 파일 다운로드 하기 

In [0]:
files.download('데이터비교v3_2차로_vdu.srt') 